## Problem Statement - IT Support Analyst

Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

In [1]:
!pip install langchain
!pip install langchain_community
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 6.9 MB/s eta 0:00:00


In [2]:
from getpass import getpass
OPENAI_API_KEY=getpass("Enter key:-")

Enter key:-··········


In [3]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [4]:
from langchain_openai import ChatOpenAI
chatgpt=ChatOpenAI(model="gpt-4o-mini",temperature=0)

In [5]:
from pydantic import BaseModel,Field

class OutputFormat(BaseModel):
  orig_msg:str=Field(description="The original customer message")
  orig_lang:str=Field(description="Detected language of the customer message e.g. Spanish")
  category:list=Field(description="1-2 word describing the category of the problem")
  trans_msg:str=Field(description="Translated customer message in English")
  response:str=Field(description="Response to the customer in orig_lang")
  trans_response:str=Field(description="Response to the customer in English")

In [6]:
from langchain_core.output_parsers import JsonOutputParser
parser=JsonOutputParser(pydantic_object=OutputFormat)

In [7]:
prompt_text="""
You are an experienced IT support agent.

Your task is to process the following customer IT support ticket message and provide a structured response based on the given output format instructions.

Customer IT Ticket:
{input_ticket}

Output Format Instructions:
{output_instructions}

Make sure to translate the message and response if needed, detect the language, and give a valid JSON response that matches the required structure.
"""

In [8]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

In [9]:
from langchain_core.prompts import PromptTemplate
template=PromptTemplate(
    template=prompt_text,
    input_variables=["input_ticket"],
    partial_variables={"output_instructions":parser.get_format_instructions}
)

In [10]:
chain=template | chatgpt

In [12]:
inputs=[{"input_ticket":input} for input in it_support_queue]

In [13]:
responses=chain.map().invoke(inputs)

In [20]:
for ticket,resp in zip(it_support_queue,responses):
  print(f"input ticket :- {ticket}\n\n")
  print(f"Response:- \n{resp.content}\n\n\n\n")

input ticket :- Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.


Response:- 
```json
{
  "orig_msg": "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
  "orig_lang": "Portuguese",
  "category": ["synchronization", "contacts"],
  "trans_msg": "I cannot sync my contacts with the phone. I always receive a failure message.",
  "response": "Por favor, verifique se você está conectado à internet e tente novamente. Se o problema persistir, tente reiniciar o telefone e verificar as configurações de sincronização.",
  "trans_response": "Please check if you are connected to the internet and try again. If the problem persists, try restarting the phone and checking the sync settings."
}
```




input ticket :- Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.


Response:- 
```json
{
  "orig_msg": "Ho problemi a stampare i documenti da remoto. Il lavoro non viene i

In [31]:
import json
import pandas as pd

def clean_json_response(resp):
    content = resp.content.strip().strip("`").strip()
    if content.startswith("json"):
        content = content[len("json"):].strip()
    return json.loads(content)

data = pd.DataFrame([clean_json_response(resp) for resp in responses])


In [32]:
data

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,"[synchronization, contacts]",I cannot sync my contacts with the phone. I al...,"Por favor, verifique se você está conectado à ...",Please check if you are connected to the inter...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,"[Printing, Remote Access]",I have problems printing documents remotely. T...,"Per risolvere il problema, ti consiglio di con...","To resolve the issue, I recommend checking the..."
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,"[Printer, Quality]","I replaced the printer toner, but the print qu...",トナーを交換した後に印刷品質が低下することがあります。まず、トナーが正しく取り付けられている...,It is possible for print quality to decrease a...
3,"Я не могу войти в систему учета времени, появл...",Russian,[Login Issue],"I cannot log into the time tracking system, an...","Пожалуйста, проверьте свои учетные данные и уб...",Please check your credentials and ensure you a...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,"[Internet, Connectivity]",My internet connection is very slow and someti...,"Merhaba, internet bağlantınızın yavaşlaması ve...","Hello, I understand the issue with your intern..."
5,Не могу установить обновление безопасности. По...,Russian,"[Update Issue, Error Code]",I cannot install the security update. An error...,"Пожалуйста, попробуйте выполнить следующие шаг...",Please try the following steps: 1. Make sure y...
